In [93]:
import pandas as pd
import numpy as np
import glob
from uimfpy.UIMFReader import *
import matplotlib.pyplot as plt
import pickle

# select the highest peaks and compare with manual curations

In [94]:
ppm = 50
th_peak_area = 1000
th_isotope_lc_cosine = 0.85
masking_threshold = 0.3 # masking threshold (e.g. 0.1 means cut-off by 10% of local maximum)
frames_for_peak=None
scans_for_peak=None

data_folder = '/Users/leej324/Downloads/LC_SLIM/'
dfs = []
concen = 5000
uimf = data_folder+'LC_{0}_heavy_repA_1_MinInt5_DComp5.uimf'.format(concen)
xic2d_file = data_folder+'LC_{0}_heavy_repA_1_MinInt5_DComp5_50ppm.pkl'.format(concen)
if (frames_for_peak is None) | (scans_for_peak is None):
    peak_out_file = data_folder+ \
            'LC_{0:d}_heavy_repA_1_MinInt5_DComp5_heavy_peaks_{1:d}ppm_area{2:d}_cosine{3:.2f}_mask{4:.1f}.csv'.format(
                concen, ppm, th_peak_area, th_isotope_lc_cosine, masking_threshold)
else:
    peak_out_file = data_folder+ \
            'LC_{0:d}_heavy_repA_1_MinInt5_DComp5_heavy_peaks_{1:d}ppm_area{2:d}_cosine{3:.2f}_f{4:d}_s{5:d}.csv'.format(
                concen, ppm, th_peak_area, th_isotope_lc_cosine, frames_for_peak, scans_for_peak)
    
df = pd.read_csv(peak_out_file, index_col=0)
df['concentration'] = concen

In [95]:
print('uimf:',uimf)
uimf_data = UIMFReader(uimf, TIC_threshold=0)

uimf: /Users/leej324/Downloads/LC_SLIM/LC_5000_heavy_repA_1_MinInt5_DComp5.uimf
####################################################################################################
# Average TOF length: 812720.0 (0.0)
# M/Z calibrators by params: {('0.347577', '0.0549547', '1.0'): <uimfpy.MzCalibrator.MzCalibrator object at 0x1209a1d30>}
####################################################################################################


In [96]:
df['scan_ms'] = uimf_data.drift_ms(df.scan)
df['scan_start_ms'] = uimf_data.drift_ms(df.scan_start)
df['scan_end_ms'] = uimf_data.drift_ms(df.scan_end)

In [97]:
df.head()

,M+1_area,M+1_cosine,M+2_area,M+2_cosine,M-1_area,M-1_cosine,frame,frame_end,frame_start,peak_area,...,scan_start,pepseq,mono_mass,iso_mass,iso,z,concentration,scan_ms,scan_start_ms,scan_end_ms
0,1383,0.903327,415138,0.896228,67,0.376444,133,145,128,398052,...,492,TLEPVKPPTVPNDY[+80]MTSPAR[+10],1152.054478,1152.054478,0,2,5000,437.24336,399.85824,448.62144
1,144913,0.994192,73075,0.986201,142673,0.991668,1471,1486,1462,199270,...,532,TLEPVKPPTVPNDY[+80]MTSPAR[+10],1152.054478,1152.054478,0,2,5000,464.87584,432.36704,483.56840
2,105614,0.990739,70734,0.991226,2174,0.885022,721,730,715,74441,...,554,TLEPVKPPTVPNDY[+80]MTSPAR[+10],1152.054478,1152.054478,0,2,5000,456.74864,450.24688,463.25040
3,27563,0.985338,13410,0.969942,111245,0.984417,173,179,167,63292,...,413,TLEPVKPPTVPNDY[+80]MTSPAR[+10],1152.054478,1152.054478,0,2,5000,341.34240,335.65336,347.03144
4,643903,0.932633,275406,0.992700,251161,0.993650,799,808,792,1151166,...,443,TLEPVKPPTVPNDY[+80]MTSPAR[+10],768.372078,768.372078,0,3,5000,383.60384,360.03496,390.91832


In [115]:
df[df.pepseq=='YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10]']

,M+1_area,M+1_cosine,M+2_area,M+2_cosine,M-1_area,M-1_cosine,frame,frame_end,frame_start,peak_area,...,scan_start,pepseq,mono_mass,iso_mass,iso,z,concentration,scan_ms,scan_start_ms,scan_end_ms
4152,940220,0.999624,393997,0.994565,1884,0.690460,1501,1506,1493,715939,...,333,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],1205.542267,1205.542267,0,2,5000,341.34240,270.63576,350.28232
4153,235534,0.997907,158387,0.992528,11972,0.960265,575,581,568,211215,...,541,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],1205.542267,1205.542267,0,2,5000,446.18328,439.68152,452.68504
4154,155714,0.997145,88724,0.981221,140527,0.998365,1406,1413,1400,183266,...,556,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],1205.542267,1205.542267,0,2,5000,457.56136,451.87232,463.25040
4155,118536,0.995011,59865,0.984520,132338,0.997649,902,908,895,158021,...,559,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],1205.542267,1205.542267,0,2,5000,459.18680,454.31048,465.68856
4156,47791,0.993107,13781,0.983040,173829,0.994261,1611,1617,1605,115026,...,523,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],1205.542267,1205.542267,0,2,5000,430.74160,425.05256,437.24336
4157,339168,0.998924,175591,0.995891,328793,0.998599,901,908,895,466973,...,380,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],804.030603,804.030603,0,3,5000,331.58976,308.83360,338.90424
4158,381974,0.999042,338529,0.999234,27881,0.845226,575,581,568,324593,...,446,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],804.030603,804.030603,0,3,5000,369.78760,362.47312,377.10208
4159,123326,0.997892,53749,0.983918,112008,0.999110,902,906,898,172617,...,322,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],804.030603,804.030603,0,3,5000,268.19760,261.69584,282.01384
4160,48982,0.987075,21625,0.973719,49140,0.989607,1405,1411,1399,68674,...,363,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],804.030603,804.030603,0,3,5000,308.83360,295.01736,316.14808
4161,31785,0.985561,18535,0.857441,46402,0.954657,406,413,399,47511,...,428,YLC[+57]EGTESPY[+80]QTGQLHPAIR[+10],804.030603,804.030603,0,3,5000,363.28584,347.84416,368.97488


In [105]:
top_peptides[top_peptides.pepseq=='AIGNNSATS[+80]PR[+10]']

,M+1_area,M+1_cosine,M+2_area,M+2_cosine,M-1_area,M-1_cosine,frame,frame_end,frame_start,peak_area,...,scan_start,pepseq,mono_mass,iso_mass,iso,z,concentration,scan_ms,scan_start_ms,scan_end_ms
4551,6135435,0.998239,1525493,0.982753,20494,0.727722,1719,1737,1704,4930131,...,384,AIGNNSATS[+80]PR[+10],1177.523661,1177.523661,0,1,5000,340.52968,312.08448,367.34944
4559,278644,0.998918,113816,0.964893,6863,0.941434,787,792,781,256826,...,380,AIGNNSATS[+80]PR[+10],589.265469,589.265469,0,2,5000,324.27528,308.83360,331.58976


In [53]:
784236*0.9

705812.4

In [101]:
top_peptides = df[df['M+1_area']>=df['M+2_area']*0.9].sort_values(by='peak_area', ascending=False).drop_duplicates(subset=['pepseq','z'])

In [110]:
manual_curation_file = data_folder + '5000fmol_25July2019_ALL_PEPTIDES_cc_gn.xlsx'
manual_curation = pd.read_excel(manual_curation_file, sheet_name='heavy')

In [111]:
# del manual_curation['Unnamed: 16']
manual_curation.head()

,Precursor Charge,Min Start Time,Max End Time,Precursor Mz,Isotope Label Type,Modified Sequence,SRM Skyline RT,SLIM Skyline RT,Charge state,m/z (heavy),m/z (light),Actual RT (min),RT (frames),IMS (ms),Width (ms),Light (Y/N),Unnamed: 16
0,1.0,123.74,125.74,2138.006180,heavy,AAAS[+80]VT[+80]PPGSLELLQPGFSK[+8],17.849433,15.83,2.0,1069.506590,1065.499491,111.514139,1115.141389,469.0,27,NaN,NaN
1,1.0,119.85,121.85,2058.039849,heavy,AAAS[+80]VTPPGSLELLQPGFSK[+8],17.297228,15.20,2.0,1029.523424,1025.516325,107.076116,1070.761157,451.0,29,Y,NaN
2,1.0,120.96,122.96,2058.039849,heavy,AAASVT[+80]PPGSLELLQPGFSK[+8],17.454798,15.70,2.0,1029.523424,1025.516325,110.598356,1105.983564,445.0,19,NaN,NaN
3,1.0,116.89,118.89,2058.039849,heavy,AAASVTPPGS[+80]LELLQPGFSK[+8],16.877041,14.70,2.0,1029.523424,1025.516325,103.553875,1035.538751,445.0,24,Y,NaN
4,1.0,84.29,86.29,1911.981489,heavy,AALGVLGPEVEKEAVS[+80]PR[+10],12.249305,10.70,2.0,956.494244,951.490110,75.375950,753.759499,443.0,14,NaN,NaN


In [113]:
conds = (df['M+1_area']>=df['M+2_area']*0.9)
conds &= (df['M-1_area']<df['peak_area']*0.5)

top_peptides = df[conds].sort_values(by='peak_area', ascending=False).drop_duplicates(subset=['pepseq','z'])
merged = manual_curation.merge(top_peptides, left_on=["Modified Sequence",'Charge state'], right_on=["pepseq",'z'], how='outer')
merged.to_csv('5000fmol_25July2019_ALL_PEPTIDES_cc_gn_jl.txt', sep='\t')

In [114]:
# conds = (df['M+1_area']>=df['M+2_area']*0.9)
# conds &= (df['M-1_area']<df['peak_area']*0.5)

# top_peptides = df[conds].sort_values(by='peak_area', ascending=False).drop_duplicates(subset=['pepseq','z'])
# merged = manual_curation.merge(top_peptides, left_on=["Modified Sequence",'Charge state'], right_on=["pepseq",'z'], how='outer')
print(((merged['RT (frames)']>0)&(merged.frame>0)).sum())
merged['frame_diff'] = 100*np.abs(merged['RT (frames)']-merged.frame)/merged['RT (frames)']
print(((merged.frame_diff>10)&(merged['RT (frames)']>0)).sum())
print(((merged.frame_diff<=10)&(merged['RT (frames)']>0)).sum())

106
34
72


In [103]:
merged[(merged.frame_diff>10)&(merged['RT (frames)']>0)&(merged['RT (frames)']>0)][['RT (frames)','frame','pepseq','mono_mass','frame_diff']]

,RT (frames),frame,pepseq,mono_mass,frame_diff
9,1012.291962,787.0,AIGNNSATS[+80]PR[+10],589.265469,22.255631
12,207.107750,420.0,ALPNNTSSS[+80]PQPK[+8],714.834138,102.792990
16,572.011881,1580.0,AQIPEGDY[+80]LSYR[+10],751.333548,176.218039
17,474.798039,1556.0,AQS[+80]FPDNRQEYSDRETQLYDK[+8],1339.584128,227.718287
23,975.660660,1163.0,ASDGSFS[+80]LPLKPVISQPIHDSK[+8],1206.108522,19.201281
24,938.324909,1163.0,ASDGSFSLPLKPVISQPIHDS[+80]K[+8],1206.108522,23.944274
25,200.767717,406.0,ASDPQS[+80]PPQVSR[+10],679.802415,102.223747
39,667.816827,1147.0,AY[+80]SFC[+57]GTVEYMAPEVVNR[+10],1091.964275,71.753684
40,655.841209,1655.0,AYC[+57]GQC[+57]S[+80]ERIWGLAR[+10],958.912180,152.347669
44,200.767717,368.0,C[+57]LQLS[+80]PPSQSK[+8],666.809077,83.296401
